# CaCO3 structures

In [3]:
from ase.spacegroup import crystal

a =  4.99
c = 17.0615
caco3 = crystal('CaCO3', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.25), 
                          (0.2578, 0.0, 0.25)], 
                spacegroup='R -3 c', 
                cellpar=[a,a,c,90,90,120])

In [5]:
from ase.visualize import view
view(caco3)

In [17]:
from ase.io import read
caco3_cif = read("caco3_r3c.cif")
view(caco3_cif)

# NaNO3 structures

In [163]:
from ase.spacegroup import crystal
from ase.visualize import view
# T = 541K
a =  5.082
c = 17.31
nano3_low = crystal('NaNO', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.25), 
                          (0.2381, 0.0, 0.25)], 
                spacegroup='R -3 c', 
                cellpar=[a,a,c,90,90,120])

print "Low Temp: T = 541K ", nano3_low.info

# T = 563K

a =  5.084
c = 8.175
nano3_high = crystal('NaNO', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.5), 
                          (0.2337, 0.0, 0.5)], 
                spacegroup='R -3 m', 
                cellpar=[a,a,c,90,90,120])

print "High Temp: T = 563K ", nano3_high.info

Low Temp: T = 541K  {'spacegroup': Spacegroup(167, setting=1), 'unit_cell': 'conventional'}
High Temp: T = 563K  {'spacegroup': Spacegroup(166, setting=1), 'unit_cell': 'conventional'}


In [239]:
view(nano3_low)

In [4]:
view(nano3_high)


# Create pair list

In [164]:
import numpy as np
import sys
sys.path.append("/home/ntm/software/PairList")

from pairlist import pairlist

In [165]:
def get_pairlist(elementA,  cutoff, atoms, elementB=None):
    cell = atoms.cell
    
    As = list()
    Bs = list()
    As_idMap = dict()
    Bs_idMap = dict()
    
    for atom in atoms:
        if atom.symbol == elementA:
            As_idMap[len(As)] = atom.index
            As.append([atom.a, atom.b, atom.c])
        if elementB:
            if atom.symbol == elementB:
                Bs_idMap[len(Bs)] = atom.index
                Bs.append([atom.a, atom.b, atom.c])
                
    As = np.array(As)
    Bs = np.array(Bs)
        
    if elementB:
        pairs = pairlist(As, cutoff, cell, Bs)
    else:
        pairs = pairlist(As, cutoff, cell)
    
    if pairs.size == 0:
        print("No pairs. Exiting...")
        return
    
    def pairlist_local2global(pairs, idMap, idMap2=None):
        if not idMap2:
            idMap2 = idMap
        
        new_pairlist = list()
        for pair in pairs[pairs[:,0].argsort()]:
            id1 = int(pair[0])
            id2 = int(pair[1])
            r   = pair[2]
            new_pairlist.append([idMap[id1], idMap2[id2], r])
        
        return new_pairlist
    
    if elementB:
        pairs = pairlist_local2global(pairs, As_idMap, Bs_idMap)
    else:
        pairs = pairlist_local2global(pairs, As_idMap)
        
    return pairs

In [166]:
pairs = get_pairlist('N', 1.3, nano3_low, 'O')

In [167]:
print "NO bonds:", len(pairs)
for pair in pairs:
    print "   pair: {0:>2} {1:>2} dist: {2:04.2f}".format(pair[0], pair[1], pair[2])

NO bonds: 18
   pair:  6 14 dist: 1.21
   pair:  6 13 dist: 1.21
   pair:  6 12 dist: 1.21
   pair:  7 16 dist: 1.21
   pair:  7 17 dist: 1.21
   pair:  7 15 dist: 1.21
   pair:  8 19 dist: 1.21
   pair:  8 20 dist: 1.21
   pair:  8 18 dist: 1.21
   pair:  9 22 dist: 1.21
   pair:  9 21 dist: 1.21
   pair:  9 23 dist: 1.21
   pair: 10 26 dist: 1.21
   pair: 10 24 dist: 1.21
   pair: 10 25 dist: 1.21
   pair: 11 29 dist: 1.21
   pair: 11 27 dist: 1.21
   pair: 11 28 dist: 1.21


# Set nitrate NO bonds from 1.21 -> 1.25 using pair list

In [168]:
bond_length=1.25 
# For fix:
#   0   = fixes the 1st atom 
#   0.5 = fixes center of bond
#   1   = fixes 2nd atom

for pair in pairs:
    nano3_low.set_distance(pair[0], pair[1], bond_length, fix=0, mic=True)

In [169]:
pairs = get_pairlist('N', 1.3, nano3_low, 'O')

In [170]:
print "NO bonds:", len(pairs)
for pair in pairs:
    print "   pair: {0:>2} {1:>2} dist: {2:04.2f}".format(pair[0], pair[1], pair[2])

NO bonds: 18
   pair:  6 14 dist: 1.25
   pair:  6 13 dist: 1.25
   pair:  6 12 dist: 1.25
   pair:  7 16 dist: 1.25
   pair:  7 17 dist: 1.25
   pair:  7 15 dist: 1.25
   pair:  8 19 dist: 1.25
   pair:  8 20 dist: 1.25
   pair:  8 18 dist: 1.25
   pair:  9 22 dist: 1.25
   pair:  9 21 dist: 1.25
   pair:  9 23 dist: 1.25
   pair: 10 26 dist: 1.25
   pair: 10 24 dist: 1.25
   pair: 10 25 dist: 1.25
   pair: 11 29 dist: 1.25
   pair: 11 27 dist: 1.25
   pair: 11 28 dist: 1.25


# Create molecules from pair list

In [248]:
molecule_ids = [None]*len(nano3_low)
molecule_counter = 0
for atomI in nano3_low:
    print atomI.symbol,
    list1 = [pair[1] for pair in pairs if atomI.index == pair[0]]
    list2 = [pair[0] for pair in pairs if atomI.index == pair[1]]
    bonded_list = list1 + list2
    if bonded_list:
        assigned = [ molecule_ids[i] for i in bonded_list+[atom.index]]
        
        if not any(assigned):
            print 'path 1', bonded_list, assigned
            for i in bonded_list+[atom.index]:
                molecule_ids[i] = molecule_counter
            molecule_counter += 1
        else:
            print 'path 2', bonded_list, assigned
            molecule_id = set(uid for uid in assigned if uid is not None)
            assert(len(molecule_id) == 1)
            for i in bonded_list+[atom.index]:
                molecule_ids[i] = list(molecule_id)[0]
    else:
        print 'path 3',
        if not molecule_ids[atomI.index]:
            molecule_ids[atomI.index] = molecule_counter
        molecule_counter += 1
    print molecule_counter, molecule_ids

Na path 3 1 [0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Na path 3 2 [0, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Na path 3 3 [0, 1, 2, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Na path 3 4 [0, 1, 2, 3, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Na path 3 5 [0, 1, 2, 3, 4, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Na path 3 6 [0, 1, 2, 3, 4, 5, None, None, None, None, None, None, None, None, N

In [238]:
for atomI in nano3_low:
    print atomI.index, molecule_ids[atomI.index]

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 6
8 6
9 6
10 6
11 6
12 6
13 6
14 6
15 6
16 6
17 6
18 6
19 6
20 6
21 6
22 6
23 6
24 6
25 6
26 6
27 6
28 6
29 6


# NaNO3 MD model

In [5]:
class McDonaldNO3(object):
    def __init__(self, supercell=[1,1,1], cif_file=None):
        self.description = "Sodim Nitrate model for crystalline odered-disordered transition"
        self.citation = { "authors" : [ {"Last" : "Lynden-Bell", "First" : "R M"},
                                        {"Last" : "Ferrario", "First" : "M "},
                                        {"Last" : "McDonald", "First" : "I R"} ],
                          "journal" : "Journal of Physics: Condensed Matter",
                          "year" : 1989,
                          "volume" : 1,
                          "pages" : 6523-6542
                        }
        self._supercell = supercell
        
        self.mol_ids = list()
        self.molecule_counter = 0
        
        if cif_file:
            self._unit_cell = read(cif_file)
        else:
            self._unit_cell = nano3_low = crystal('NaNO', [(0.0,0.0,0.0), 
                                                           (0.0, 0.0, 0.25), 
                                                           (0.2381, 0.0, 0.25)], 
                                             spacegroup='R -3 c', 
                                             cellpar=[a,a,c,90,90,120])
        def _makeSuperCell(self):
            lat = self._lat
            sc  = self._supercell
            self.system = Atoms(cell=[sc[0]*lat,sc[1]*lat,sc[2]*lat], pbc=[1,1,1])
            
    def lammpsParameters(self):
        # Simulation Parameters

        eps_ff = 70.60 # units: K
        sig_ff =  2.70 # units: Angstrom

        eps_ff *= units.kB / (units.kcal/units.mol) # units: kcal/mol

        #print(eps_ff,sig_ff)

        # LAMMPS input
        header = ['units real',
                  'atom_style molecular',
                  'bond_style harmonic',
                  'atom_modify map array sort 0 0']

        cmds = ["pair_style lj/cut 10.0",
                "pair_coeff * * 0.0 0.0",
                "pair_coeff 2 2 "+str(eps_ff)+' '+str(sig_ff),
                "bond_coeff * 100.0 1.565",
                "fix 1 all nve",
                "fix 1 all rigid/nve/small molecule"]

        return header, cmds


In [6]:
from ase.calculators.lammpslib import write_lammps_data
nano3 = NaNO3(supercell=[1,1,1])
header, cms = nano3.lammpsParameters()
system = nano3
write_lammps_data('config.data',
                  molecules,
                  {'Na': 1, 'N': 2, 'O': 3},
                  cutoff=,
                  bond_types=[(7,8)],
                  angle_types=[(8,7,8)],
                  units='real')

SyntaxError: invalid syntax (<ipython-input-6-dd2a6b327c64>, line 8)

In [2]:
nano3_low


NameError: name 'nano3_low' is not defined